In [1]:
import numpy as np

from pose_skeletons import get_skeleton_def

from bvhtoolbox import Bvh, BvhNode, BvhTree, get_affines
import transforms3d as t3d

from ipywidgets import Layout, IntSlider, interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from k3d.transform import process_transform_arguments
import k3d

import math

/Users/d.p.saakes/code/pose_skeletons_clone/.venv/lib/python3.12/site-packages/bvhtoolbox/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
with open("data/umu_002.bvh") as f:
    # mocap = Bvh(f.read())
    mocap = BvhTree(f.read())

In [4]:
def world_joint_positions(bvh_tree, scale=1.0, end_sites=False):

    time_col = np.arange(0, (bvh_tree.nframes - 0.5) * bvh_tree.frame_time, bvh_tree.frame_time)[:, None]
    root = next(bvh_tree.root.filter('ROOT'))

    bvh_dict = {}
    
    def get_world_positions(joint):
        if joint.value[0] == 'End':
            joint.world_transforms = np.tile(t3d.affines.compose(np.zeros(3), np.eye(3), np.ones(3)),
                                             (bvh_tree.nframes, 1, 1))
        else:
            channels = bvh_tree.joint_channels(joint.name)
            axes_order = ''.join([ch[:1] for ch in channels if ch[1:] == 'rotation']).lower()  # FixMe: This isn't going to work when not all rotation channels are present
            axes_order = 's' + axes_order[::-1]
            joint.world_transforms = get_affines(bvh_tree, joint.name, axes=axes_order)            
        if joint != root:
            # For joints substitute position for offsets.
            offset = [float(o) for o in joint['OFFSET']]
            joint.world_transforms[:, :3, 3] = offset
            joint.world_transforms = np.matmul(joint.parent.world_transforms, joint.world_transforms)
        if scale != 1.0:
            joint.world_transforms[:, :3, 3] *= scale

        axis = [1, 0, 0]  # X-axis
        angle = math.pi / 2  # 90 degrees    
        # Create the 4x4 affine transformation matrix
        # (We provide no translation, so [0,0,0])
        R_y_to_z = t3d.axangles.axangle2mat(axis, angle)

        pos = joint.world_transforms[:, :3, 3]
        pos_zup = pos @ R_y_to_z.T

        print(joint.name)

        bvh_dict[joint.name] = pos_zup # pos # r.apply(pos)                 
        if end_sites:
            end = list(joint.filter('End'))
            if end:
                get_world_positions(end[0])  # There can be only one End Site per joint.
        for child in joint.filter('JOINT'):
            get_world_positions(child)
    
    get_world_positions(root)
   
    return bvh_dict

world_coordinates = world_joint_positions(mocap)

Hips
Spine
Spine1
Neck
Head
LeftShoulder
LeftArm
LeftForeArm
LeftHand
LeftHandThumb1
LeftHandThumb2
LeftHandThumb3
LeftHandIndex1
LeftHandIndex2
LeftHandIndex3
LeftHandMiddle1
LeftHandMiddle2
LeftHandMiddle3
LeftHandRing1
LeftHandRing2
LeftHandRing3
LeftHandPinky1
LeftHandPinky2
LeftHandPinky3
RightShoulder
RightArm
RightForeArm
RightHand
RightHandThumb1
RightHandThumb2
RightHandThumb3
RightHandIndex1
RightHandIndex2
RightHandIndex3
RightHandMiddle1
RightHandMiddle2
RightHandMiddle3
RightHandRing1
RightHandRing2
RightHandRing3
RightHandPinky1
RightHandPinky2
RightHandPinky3
LeftUpLeg
LeftLeg
LeftFoot
LeftToeBase
RightUpLeg
RightLeg
RightFoot
RightToeBase


In [15]:
optitrack_def = get_skeleton_def("optitrack")

[optitrack_def.l_shoulder, optitrack_def.r_shoulder]


[<OptitrackJoints.LeftArm: 6>, <OptitrackJoints.RightArm: 25>]

In [26]:
def set_frame(n):
    n = min(mocap.nframes-1, n)
    co = []              
    for joint in mocap.get_joints():
        co.append( world_coordinates[joint.name][n] )    
    joints_plot.positions = co
    
    standard_joints_plot.positions = np.array(co)[ [optitrack_def.l_shoulder, optitrack_def.r_shoulder] ]
    
    lines_plot.vertices = co

plot = k3d.plot() # grid_visible=False)
joints_plot = k3d.points([], point_size=0.005, shader='3d', color=0xff0000) 
standard_joints_plot = k3d.points([], point_size=0.08, shader='3d', color=0x0000ff) 

lines_plot = k3d.lines([], list(optitrack_def.bones), width=0.005, indices_type="segment")

plot += joints_plot
plot += standard_joints_plot
plot += lines_plot
plot.display()

interact(set_frame, n=widgets.IntSlider(min=0, max=mocap.nframes-1, step=1, value=10, layout=widgets.Layout(width='100%')))

Output()

interactive(children=(IntSlider(value=10, description='n', layout=Layout(width='100%'), max=7296), Output()), …

<function __main__.set_frame(n)>